## Import

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import torch
import matplotlib.pyplot as plt

In [3]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [4]:
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Load model

In [5]:
%run ../models/report_generation/__init__.py
%run ../models/checkpoint/__init__.py
%run ../utils/files.py

In [6]:
# run_name = '0716_211601_lstm-att_lr0.0001_densenet-121'
# run_name = '0115_175006_h-lstm-att-v2_lr0.001_satt_densenet-121-v2_noes'
# run_name = '0115_064249_h-lstm-att-v2_lr0.001_densenet-121-v2_noes_front'
# run_name = '0401_222625_mimic-cxr_lstm-v2_lr0.0001_mobilenet-v2_size256'

# run_id = RunId('0420_031015', False, 'rg', 'best-cnn')
run_id = RunId('0426_143345', False, 'rg')

In [9]:
compiled_model = load_compiled_model_report_generation(run_id, device=DEVICE)
_ = compiled_model.model.eval()
compiled_model.metadata['decoder_kwargs']

{'decoder_name': 'h-lstm-v2',
 'vocab_size': 1813,
 'embedding_size': 100,
 'hidden_size': 100,
 'features_size': 1024,
 'teacher_forcing': True,
 'dropout_recursive': 0,
 'dropout_out': 0}

In [10]:
VOCAB = compiled_model.metadata['vocab']
len(VOCAB)

1813

In [11]:
%run ../training/report_generation/flat.py
%run ../training/report_generation/hierarchical.py

In [13]:
decoder_name = compiled_model.metadata['decoder_kwargs']['decoder_name']
HIERARCHICAL = is_decoder_hierarchical(decoder_name)
if HIERARCHICAL:
    create_dataloader = create_hierarchical_dataloader
else:
    create_dataloader = create_flat_dataloader
HIERARCHICAL

True

## Load data

In [14]:
%run ../datasets/__init__.py
%run ../utils/nlp.py

In [16]:
model_dataset_kwargs = compiled_model.metadata['dataset_kwargs']
dataset_kwargs = {
    'create_dataloader_fn': create_dataloader,
    'dataset_name': 'iu-x-ray',
    'max_samples': None,
    'frontal_only': True,
    'image_size': (256, 256),
    'vocab': VOCAB,
    'batch_size': 10,
    'vocab_greater': model_dataset_kwargs.get('vocab_greater', None),
    'reports_version': model_dataset_kwargs.get('reports_version', LATEST_REPORTS_VERSION),
}

train_dataloader = prepare_data_report_generation(dataset_type='train', **dataset_kwargs)
val_dataloader = prepare_data_report_generation(dataset_type='val', **dataset_kwargs)
test_dataloader = prepare_data_report_generation(dataset_type='test', **dataset_kwargs)
len(train_dataloader.dataset), len(val_dataloader.dataset), len(test_dataloader.dataset)

(3031, 385, 384)

## Eval

In [17]:
from captum.attr import visualization
# from skimage.color import rgb2gray, gray2rgb
# from skimage.transform import resize

In [18]:
%run ../training/report_generation/hierarchical.py
%run ../utils/common.py
%run ../utils/nlp.py

In [19]:
report_reader = ReportReader(VOCAB)

In [20]:
def eval_sample(image, report,
                show=True, device=DEVICE, free=False, **kwargs):
    # Prepare inputs
    images = image.unsqueeze(0).to(device)
    if HIERARCHICAL:
        reports = split_sentences_and_pad(report)
    else:
        reports = torch.tensor(report)

    reports = reports.unsqueeze(0).to(device)
    
    # Pass thru model
    if not HIERARCHICAL:
        del kwargs['max_sentences']
    tup = compiled_model.model(images, reports, free=free, **kwargs)
    
    # Parse outputs
    if HIERARCHICAL:
        generated = _flatten_gen_reports(tup[0], tup[1])
    else:
        generated = tup[0]
        _, generated = generated.max(dim=-1)

    generated = generated.squeeze(0).cpu()
    
    # Print result
    original_report = report_reader.idx_to_text(report)
    generated_report = report_reader.idx_to_text(generated)
    if show:
        print('GROUND TRUTH:\n', original_report)
        print('-'*20)
        print('GENERATED:\n', generated_report)
        
    return original_report, generated_report, tup

In [21]:
dataset = train_dataloader.dataset

In [26]:
idx = 7

item = dataset[idx]
image = item.image
report = item.report

gt, gen, other = eval_sample(image, report,
                             free=True, max_sentences=100, max_words=40)

GROUND TRUTH:
 no active disease .
--------------------
GENERATED:
 no acute cardiopulmonary abnormality identified . . . . . . . . . . process . . process . process . process . tumor for malignancy . dysmorphic removed three maintained maintained maintained . abnormalities . . artifact .


### Hierarchical case

In [ ]:
stops = other[1].detach().cpu()
# stops = stops[0]
print(stops.size())
stops

In [ ]:
out_words = other[0].detach().squeeze(0)
out_words = out_words.argmax(dim=-1)
out_words.size()

In [ ]:
scores = other[2].detach().squeeze(0).cpu()
scores.size()

In [ ]:
image_color = image.detach().permute(1, 2, 0).cpu().numpy()
image_color = arr_to_range(image_color)

image_color.shape

In [ ]:
sentence_idx = 1
heatmap = scores[sentence_idx].numpy()
heatmap = gray2rgb(heatmap)
heatmap = resize(heatmap, image_color.shape)
heatmap.shape

In [ ]:
report_reader.idx_to_text(out_words[sentence_idx])

In [ ]:
visualization.visualize_image_attr_multiple(heatmap,
                                            image_color,
                                            methods=['original_image',
                                                     'blended_heat_map'],
                                            signs=['all', 'positive'],
                                            cmap='jet',
                                            show_colorbar=True,
                                           )

## Debug report-reader

In [ ]:
%run ../utils/nlp.py

In [ ]:
report_reader = ReportReader(compiled_model.metadata['vocab'], ignore_pad=True)

In [ ]:
idx = 20

item = train_dataset[idx]
images = item.image
images = item.image.unsqueeze(0).to(DEVICE)
reports = split_sentences_and_pad(item.report)

out = compiled_model.model(images, reports, free=False)
out = out[0].argmax(dim=-1).squeeze(0).detach().cpu()
out.size()

In [ ]:
out

In [ ]:
report_reader.idx_to_text(out)

In [ ]:
out.tolist()